In [1]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
import torch
import matplotlib.pyplot as plt
import numpy as np
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from d2l import torch as d2l
from torchvision import  transforms


# data = unpickle('./CIFAR10/cifar-10-batches-py/test_batch')
#
# data[b'data'][0] # array([158, 159, 165, ..., 124, 129, 110], dtype=uint8)

## Ret50

In [2]:

# 1x1 conv -> 3x3 conv -> 1x1 conv
class Bottleneck(nn.Module):
    def __init__(self, in_channels, channels, stride=1, use_1x1conv=False):
        super(Bottleneck,self).__init__()
        self.conv1 = nn.Conv2d(in_channels, channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)
        self.conv3 = nn.Conv2d(channels, channels*4, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(channels*4)

        if use_1x1conv:
            self.conv4 = nn.Conv2d(
                in_channels, channels*4, kernel_size=1, stride=stride
            )
        else:
            self.conv4 = None

    def forward(self, x):
        # 1x1 conv 通道数：in_channels -> channels
        out = F.relu(self.bn1(self.conv1(x)))
        # 3x3 conv 通道数：channels -> channels
        out = F.relu(self.bn2(self.conv2(out)))
        # 1x1 conv 通道数: channels -> 4*channels
        out = self.bn3(self.conv3(out))

        # 恒等映射 or 1x1 conv
        if  self.conv4 == None:
            identity = x
        else:
            identity = self.conv4(x)


        out += identity
        return F.relu(out)
def bottleneck_block(in_channels, channels, num_bottlenecks, not_FirstBlock = True):
    # 第一个neck使用1x1conv，剩余的neck不使用1x1conv
    # 第一个block的stride=1，后面的block的stride=2
    blk = []
    for i in range(num_bottlenecks):
        if i == 0:
            blk.append(
                Bottleneck(in_channels, channels, stride=not_FirstBlock+1, use_1x1conv=True)
            )
        else:
            blk.append(
                Bottleneck(channels*4, channels)
            )
    return blk
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b2 = nn.Sequential(*bottleneck_block(64, 64, 3, not_FirstBlock=False))
b3 = nn.Sequential(*bottleneck_block(64*4, 128, 3))
b4 = nn.Sequential(*bottleneck_block(128*4, 256, 3))
b5 = nn.Sequential(*bottleneck_block(256*4, 512, 3))
ret50 = nn.Sequential(
    b1, b2, b3, b4, b5,
    nn.AdaptiveAvgPool2d((1,1)),
    nn.Flatten(),
    nn.Linear(2048, 10)
)

## 使用28*28像素 进行训练分类

In [4]:
trans24 = transforms.Compose(
    [transforms.ToTensor()])#28*28
train_dataset = datasets.FashionMNIST(
    root=r'./', train=True, transform=trans24, download=True)
test_dataset = datasets.MNIST(
    root=r'./', train=False, transform=trans24, download=True)

# 配置数据加载器
batch_size = 256
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size, shuffle=True)

## 训练完成后会保存模型，可以修改模型的保存路径。

# 保存和加载整个模型
torch.save(net, 'net.pth')
model = torch.load('net.pth')

# 将my_resnet模型储存为my_resnet.pth
torch.save(net.state_dict(), "net_parameter.pth")
# 加载resnet，模型存放在my_resnet.pth
net.load_state_dict(torch.load("net_parameter.pth"))


In [5]:
# def train(net, train_iter, test_iter, epochs,scheduler=None, device):


# device = d2l.try_gpu()
device = torch.device("cuda:0")
def train_loss(net, train_iter, test_iter, epochs, loss,  device, lr):
# 参数初始化1
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            # nn.init.xavier_uniform_(m.weight )#80.87%
            nn.init.kaiming_uniform_(m.weight)
    net.apply(init_weights)

    # net.load_state_dict(torch.load("./resnet18-f37072fd.pth"),strict=False)


    net.to(device)#.to(device) 可以指定CPU 或者GPU
    animator = d2l.Animator(xlabel='epoch', xlim=[0, epochs],#画出loos新添加画图工具
                            legend=['train loss', 'train acc', 'test acc'])

    trainer = torch.optim.Adam(net.parameters(), lr=lr)


    for epoch in range(epochs):
        # 训练损失之和，训练准确率之和，样本数
        metric = d2l.Accumulator(3)
        net.train()#在使用 pytorch 构建神经网络的时候，训练过程中会在程序上方添加一句model.train()，作用是 启用 batch normalization 和 dropout 。
        timer, num_batches = d2l.Timer(), len(train_iter)
        for i, (X, y) in enumerate(train_iter):

            trainer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])

            train_loss = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            # 画出loos新添加
            if (i + 1) % 50 == 0:
                animator.add(epoch + i / len(train_iter),
                             (train_loss, train_acc, None))

            if (i + 1) % (num_batches // 30) == 0 or i == num_batches - 1:
                print(f'Epoch: {epoch+1}, Step: {i+1}, Loss: {train_loss:.4f}')

        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
        # 画出loos新添加
        animator.add(epoch+1, (None, None, test_acc))

        #新添加 学习率调度器 多因子调度器


    print(f'train loss {train_loss:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    torch.save(net.state_dict(),
               f".\\model\\ResNet-18_CIFAR-10_Epoch{epochs}_Accuracy{test_acc*100:.2f}%.pth")
    torch.save(net,f".\\model\\net_CIFAR-10_Epoch{epochs}_Accuracy{test_acc*100:.2f}%.pth")


In [ ]:
def train(net, train_iter, test_iter, epochs, lr, device):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            # nn.init.xavier_uniform_(m.weight )#80.87%d2l使用的
            nn.init.kaiming_uniform_(m.weight)
    net.apply(init_weights)

    # net.load_state_dict(torch.load("./resnet18-f37072fd.pth"),strict=False)

    print(f'Training on:[{device}]')
    net.to(device)#.to(device) 可以指定CPU 或者GPU
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    timer, num_batches = d2l.Timer(), len(train_iter)

    # loss = []
    # acc  = []
    for epoch in range(epochs):
        # 训练损失之和，训练准确率之和，样本数
        metric = d2l.Accumulator(3)
        net.train()#在使用 pytorch 构建神经网络的时候，训练过程中会在程序上方添加一句model.train()，作用是 启用 batch normalization 和 dropout 。
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            # train_l.tolist()
            # train_acc.tolist()
            # loss.append(train_l)
            # acc.append(train_acc)

            if (i + 1) % (num_batches // 30) == 0 or i == num_batches - 1:
                print(f'Epoch: {epoch+1}, Step: {i+1}, Loss: {train_l:.4f}')
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
        print(            f'Train Accuracy: {train_acc*100:.2f}%, Test Accuracy: {test_acc*100:.2f}%')
    print(f'{metric[2] * epochs / timer.sum():.1f} examples/sec '
          f'on: [{str(device)}]')
    # torch.save(net.state_dict(),
    #            f".\\model\\\ResNet-18_CIFAR-10_Epoch{epochs}_Accuracy{test_acc*100:.2f}%.pth")
    # torch.save(net,f".\\model\\net_CIFAR-10_Epoch{epochs}_Accuracy{test_acc*100:.2f}%.pth")


## 在Pytorch中构建好一个模型后，一般需要进行预训练权重中加载。torch.load_state_dict()函数就是用于将预训练的参数权重加载到新的模型之中，操作方式如下所示：
# sd_net = torchvision.models.resnte50(pretrained=False)
# sd_net.load_state_dict(torch.load('*.pth'), strict=True)
在本博文中重点关注的是 属性 strict; 当strict=True,要求预训练权重层数的键值与新构建的模型中的权重层数名称完全吻合；如果新构建的模型在层数上进行了部分微调，则上述代码就会报错：说key对应不上。

此时，如果我们采用strict=False 就能够完美的解决这个问题。也即，与训练权重中与新构建网络中匹配层的键值就进行使用，没有的就默认初始化。


## 在Pytorch中构建好一个模型后，一般需要进行预训练权重中加载。torch.load_state_dict()函数就是用于将预训练的参数权重加载到新的模型之中，操作方式如下所示：
# sd_net = torchvision.models.resnte50(pretrained=False)
# sd_net.load_state_dict(torch.load('*.pth'), strict=True)
在本博文中重点关注的是 属性 strict; 当strict=True,要求预训练权重层数的键值与新构建的模型中的权重层数名称完全吻合；如果新构建的模型在层数上进行了部分微调，则上述代码就会报错：说key对应不上。

此时，如果我们采用strict=False 就能够完美的解决这个问题。也即，与训练权重中与新构建网络中匹配层的键值就进行使用，没有的就默认初始化。


In [ ]:
epochs, lr = 20, 0.001#epochs,lr = 20, 0.001
device = torch.device("cuda:0")
loss = nn.CrossEntropyLoss()
# train_loss(net, train_iter, test_iter, num_epochs, loss, trainer, device, lr
train_loss(ret50, train_loader, test_loader, epochs, loss, device, lr)

Epoch: 1, Step: 7, Loss: 5.1789
Epoch: 1, Step: 14, Loss: 3.3592
Epoch: 1, Step: 21, Loss: 2.5962
Epoch: 1, Step: 28, Loss: 2.1353
Epoch: 1, Step: 35, Loss: 1.8598
Epoch: 1, Step: 42, Loss: 1.6738
Epoch: 1, Step: 49, Loss: 1.5202
